# Spark Caching

## Устройство памяти воркера

Вся доступная память разбивается на четыре раздела:

- **зарезервировано**: область памяти, которая используется для внутренних нужд процессов воркера. Всегда равно 300 МБ, не может быть меньше или больше;
- **исполнение (execution)**: область памяти для выполнения логики запросов - сортировка, агрегация, перемешивание и пр.;
- **кеш (storage)***: область памяти доступная для хранения пользовательских объектов в кеше;
- **пользовательские объекты**: область памяти для пользовательских объектов и метаданных кластера.

![](../imgs/spark-memory-layout.drawio.svg)

## Стратегия работы кеша

Кеш хранит объекты по стратегии LRU - Least Recent Use. Так, если нужно добавить новый объект в кеш, а места там нет, то Apache Spark выбросит из кеша данные, которые не использовались дольше всего.

В кеш можно положить партицию целиком, невозможно положить в кеш только часть партиции.

![](../imgs/spark-cache-lru.drawio.svg)

### Приёмники

Вызов метода [`DataFrame#cache`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.cache.html) кеширует датафрейм в память, а партиции, которые невозможно уместить в памяти, попадают на диск. Для более тонкой настройки кеширования необходимо использовать метод [`DataFrame#persist`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.persist.html), который принимает `enum` типа [`StorageLevel`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.StorageLevel.html). Его значения:

- `DISK_ONLY` - кешировать датафрейм целиком на диск;
- `DISK_ONLY_2`- кешировать датафрейм целиком на диск в двух копиях: одна копия на локальном диске воркера, а вторая на локальном диске любого другого воркера в кластере;
- `DISK_ONLY_3` - кешировать датафрейм целиком на диск в двух копиях: одна копия на локальном диске воркера, а вторая на локальном диске любых двух других воркеров в кластере;
- `MEMORY_AND_DISK` - кешировать датафрейм в память, а все партиции, которые не поместились в память, положить на диск. Синонимом `df.persist(StorageLevel.MEMORY_AND_DISK)` является прямой вызов [`DataFrame#cache`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.cache.html);
- `MEMORY_AND_DISK_2` - кешировать датафрейм в память, а все партиции, которые не поместились в память, положить на диск. Сохранить вторую копию на любом другом воркере;
- `MEMORY_AND_DISK_DESER` - кешировать датафрейм в память, а все партиции, которые не поместились в память, положить на диск. Данные хранятся без сжатия;
- `MEMORY_ONLY` - кешировать датафрейм в память. Партиции, которые не помещаются в память, игнорировать. Если их данные потребуются в будущем, вычислить заново каждый раз;
- `MEMORY_ONLY_2` - аналогичен `MEMORY_ONLY`, но каждая партиция хранится в кеше в двух экземплярах, второй экземпляр попадает на любой другой воркер в кластере;
- `OFF_HEAP` - положить данные вне HEAP (не мешаем Garbage Collector).

## Запуск приложения

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession \
  .builder \
  .appName("Caching") \
  .master("local[4]") \
  .getOrCreate()

## Подготовка данных

In [ ]:
!mkdir -p /tmp/taxi
!unzip -o -d /tmp/taxi ./data/taxi.zip

In [ ]:
taxi_df = spark.read.parquet("/tmp/taxi")

In [ ]:
taxi_df.printSchema()

In [ ]:
taxi_df.rdd.getNumPartitions()

In [ ]:
taxi_df.groupby(F.spark_partition_id()).count().show()

# Особенности кеширования

Закэшируем запрос:

In [ ]:
taxi_cached_df = taxi_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 3) \
  .cache()

In [ ]:
taxi_cached_df.explain(True)

На плане можно увидеть, что фильтрация датасета выполняется перед тем, как датасет попадет в кеш:

![Spark Cache](../imgs/spark-cache-plan.drawio.svg)

### Ситуация 1. Поменять операции местами

Рассмотрим запрос аналогичный предыдущему, в котром операции `filter` и `select` поменялись местами:

![Query Switch operaors order](../imgs/spark-cache-operators-order.drawio.svg)

In [ ]:
taxi_select_after_filter_df = taxi_df.filter(taxi_df.passenger_count > 3) \
  .select(taxi_df.trip_distance, taxi_df.passenger_count)

**Вопрос:** удастся ли Spark воспользоваться кешем?

In [ ]:
taxi_select_after_filter_df.explain()

<details>
    <summary><strong>Ответ</strong></summary>
    <p>Не удалось.</p>
    <p><i>Cache Manager</i> вступает в игру на стадии после логического плана, но перед оптимизатором. После оптимизатора планы запроса для <code>taxi_select_after_filter_df</code> и <code>taxi_cached_df</code> будут одинаковыми. Логические планы (Analyzed Logical Plans) у них разные, поэтому кэширование не используется в работе</p>
    <img src="../imgs/spark-cache-operators-order-plan.drawio.svg" />
</details>

In [ ]:
# закешированный запрос
taxi_cached_df.explain(True)

In [ ]:
# новый запрос
taxi_select_after_filter_df.explain(True)

### Ситуация 2. Усилить условие

Рассмотрим запрос, аналогичный `taxi_cached_df`, в котором условие усилено:

![Spark Cache Stronger Condition](../imgs/spark-cache-stronger-condition.drawio.svg)

In [ ]:
taxi_stronger_condition_df = taxi_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 7)

**Вопрос**: удастся ли Spark воспользоваться кэшем?

In [ ]:
taxi_stronger_condition_df.explain()

<details>
    <summary><strong>Ответ</strong></summary>
    <p>Не удалось.</p>
    <p>Чисто логически можно заключить, что данные находятся в кэше, но Spark не будет их читать оттуда, потому что логические планы (Analyzed Logical Plans) отличаются: в этот раз условие фильтрации другое.</p>
    <p>Воспользоваться кэшем в этот раз можно следующим образом:</p>
    <img src="../imgs/spark-cache-stronger-condition-plan.drawio.svg" />
</details>

In [ ]:
taxi_stronger_condition_df2 = taxi_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 3) \
  .filter(taxi_df.passenger_count > 7)

In [ ]:
taxi_stronger_condition_df2.explain(True)

In [ ]:
taxi_cached_df.explain(True)

![Spark Stronger Condition Fixed](../imgs/spark-cache-stronger-condition-plan.drawio-fixed.svg)

### Ситуация 3. Выбрать меньше полей

Рассмотрим запрос, который выбирает меньше полей, чем `taxi_cached_df`:

![Select Fewer Field](../imgs/spark-cache-get-fewer-fields.drawio.svg)

In [ ]:
taxi_fewer_columns_df = taxi_df.select(taxi_df.trip_distance) \
  .filter(taxi_df.passenger_count > 3)

**Вопрос:** удастся ли Spark воспользоваться кэшем?

In [ ]:
taxi_fewer_columns_df.explain()

<details>
    <summary><strong>Ответ</strong></summary>
    <p>Удалось.</p>
    <p>Логически можно продолжить рассуждения о том, что логические план (Analyzed Logical Plans) отличается от плана <code>taxi_cached_df</code> - выбирается только одна колонка. Фильтрация выполняется по <code>passenger_count</code>, который отсутствует в проекции (список колонок на выборку). Spark воспользуется правилом <code>ResolveMissingReferences</code> и добавит <code>passenger_count</code> к проекции, и тогда планы <code>taxi_fewer_columns_df</code> и <code>taxi_cached_df</code> станут идентичными. <i>Cache Manager</i> найдет закешированный план и воспользуется им.</p>
    <img src="../imgs/spark-cache-get-fewer-fields-plan.drawio.svg" />
</details>

In [ ]:
taxi_fewer_columns_df.explain(True)

## Почему так?

Особенность [`CacheManager`](https://github.com/apache/spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/execution/CacheManager.scala) заключается в том, что он вступает в работу между стадиями `Analyzed Logical Plan` и `Optimized Logical Plan`, поэтому ему могут быть недоступны трансформации дерева, выполненные на стадии `Optimized Logical Plan` и далее:

![Catalyst Cache Manager](../imgs/spark-cache-manager.drawio.svg)


В качестве [ключа](https://github.com/apache/spark/blob/445c5417ea1d7b7f4fb055153eab8fb0f711a183/sql/core/src/main/scala/org/apache/spark/sql/execution/CacheManager.scala#L42) для закешированных данных Cache Manager использует план/дерево. Если Cache Manager находит поддерево в планах новых запросов, для которого есть закешированные данные, то он перенаправляет запросы в кеш.

> Ключом закешированных данных является дерево логического плана запроса


## Выводы

### 1. Создавайте новый DataFrame при кешировании другого DataFrame

Создание нового DataFrame (`taxi_cached_df`) при кэшировании позволит избежать проблем с кэшированием

```python
taxi_cached_df = taxi_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 3) \
  .cache()
```

In [ ]:
taxi_cached_df.filter(taxi_df.passenger_count > 3) \
  .select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .explain()

In [ ]:
taxi_cached_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 7) \
  .explain()

In [ ]:
taxi_cached_df.select(taxi_df.trip_distance) \
  .filter(taxi_df.passenger_count > 3) \
  .explain()

### 2. Явно исключайте из кеша данные, которые больше не нужны

При помощи [`DataFrame#unpersist()`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.unpersist.html) можно явно убрать из кэша данные. Когда кэш заполняется, Spark начинает выбрасывать из кэша данных по стратегии LRU (Least Recent Used). Явный вызов [`unpersist()`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.unpersist.html) дает больше контроля над кэшем.

Чем меньше памяти занято кэшем, тем больше памяти Spark может использовать для выполнения работы (создание служебных объектов HashMap, и т.д.)

In [ ]:
taxi_cached_df.unpersist()

### 3. Кэшируйте только необходимое

Кэширование всего датафрейма является контпродуктивным подходом, если дальнейшая выборка идет лишь по нескольким полям. В этом случае, лучше:

1. создать новый `DataFrame`, в котором выбраны только нужные колонки,
1. закешировать этот `DataFrame`,
1. выполнять выборку на базе закешированного `DataFrame`.

In [ ]:
cached_taxi_df = taxi_df.cache()

In [ ]:
passenger_count_df = cached_taxi_df.select(cached_taxi_df.passenger_count)

In [ ]:
passenger_count_df.explain()

![Read Excessive Columns From Disk](../imgs/spark-excessive-cache-data.drawio.svg)

In [ ]:
taxi_date_distance_df = cached_taxi_df.select(cached_taxi_df.tpep_pickup_datetime, cached_taxi_df.trip_distance)

In [ ]:
taxi_date_distance_df.explain()

![Excessive Caching](../imgs/spark-excessive-cache-data.drawio-2.svg)

**^ Так делать очень плохо**

**Лушее решение**: закэшировать только тот объем данных который реально будет использоваться:

In [ ]:
cached_taxi_df.unpersist()

In [ ]:
cached_taxi_df = taxi_df.select(
    cached_taxi_df.passenger_count,
    cached_taxi_df.tpep_pickup_datetime,
    cached_taxi_df.trip_distance
  ) \
  .cache()

In [ ]:
cached_taxi_df.explain()

In [ ]:
passenger_count_df = cached_taxi_df.select(cached_taxi_df.passenger_count)
passenger_count_df.explain()

In [ ]:
taxi_date_distance_df = cached_taxi_df.select(cached_taxi_df.tpep_pickup_datetime, cached_taxi_df.trip_distance)
taxi_date_distance_df.explain()

![Cache Only Needed](../imgs/spark-cache-needed-only.drawio.svg)

После окончания работы с кэшем необходимо освободить кеш:

In [ ]:
cached_taxi_df.unpersist()

## А может без кэширования быстрее?

Датафрейм `taxi_df` строится на основе parquet файла:

In [ ]:
taxi_df.explain()

Получить общее число строк:

In [ ]:
taxi_df.count()

На плане видно, что с файловой системы ни одна колонка не считывается:

![](../imgs/spark-parquet-count.drawio.svg)

Получить число строк, попадающих по фильтру:

In [ ]:
passengers_filter_taxi_df = taxi_df.filter(taxi_df.passenger_count > 3)
passengers_filter_taxi_df.explain()

**(!!!)** План запроса показывает, что данные из parquet файла отфильтровываются уже на этапе чтения (**Pushed Filters**):

![](../imgs/spark-parquet-pushed-filters.drawio.svg)

In [ ]:
passengers_filter_taxi_df.count()

Положим `taxi_df` в кеш:

In [ ]:
taxi_cached_df = taxi_df.cache()

In [ ]:
taxi_cached_df.explain()

Первая опреация работает долго, т.к. нужно положить данные в кэш:

In [ ]:
taxi_cached_df.count()

In [ ]:
taxi_cached_df.count()

Получить число строк из датафрейма в кэше, попадающих по фильтру:

In [ ]:
passengers_filter_taxi_cached_df = taxi_cached_df.filter(taxi_cached_df.passenger_count > 3)
passengers_filter_taxi_cached_df.explain()

План запроса показывает, что данные из parquet читаются целиком, а фильтрация происходит на более поздней страдии:


![](../imgs/spark-parquet-pushed-filters-index-scan.drawio.svg)

In [ ]:
taxi_cached_df.unpersist()

### Выводы

1. Формат Parquet позволяет провести оптимизацию, при которой можно не читать ненужные данные из файла, если известно условие фильтрации;
1. Кэширование вынуждает Spark прочитать весь parquet файл целиком;
1. Кэширование отрезает Spark возможность применения оптимизации Pushed Filters;
1. Все встроенные источники данных (`json`, `csv`, `parquet`, `orc`, `avro`) поддерживают правило оптимизации `Pushed Filters`.

### Задание

1. Какой запрос будет работать эффективнее (быстрее, меньше ресурсов и т.д.):

```python
taxi_df.select(passenger_count)
```
или
```
cached_taxi_df = taxi_df.cache()
cached_taxi_df.select(passenger_count)
```
?

Игнорировать затраты необходимые для добавления данных в кеш.

2. Какое правило оптимизации включается в работу?

## Кэширование в SQL

Можно выполнять кэширование таблиц при работе со Spark через датафреймы.

In [ ]:
taxi_df.createOrReplaceTempView("taxi")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("SELECT count(*) FROM taxi").show()

Инструкция `cache` позволяет закэшировать таблицу:

In [ ]:
spark.sql("cache table taxi")

In [ ]:
spark.sql("SELECT count(*) FROM taxi").show()

Инструкция `uncache` позволяет убрать таблицу из кэша:

In [ ]:
spark.sql("uncache table taxi")

Можно заметить, что в отличии от Spark DSL, кэширование в Spark SQL выполняется сразу, тогда как `cache()` метод в Spark DSL ленивая операция. Можно сделать операцию кэшировани в Spark SQL также ленивой:

In [ ]:
spark.sql("cache lazy table taxi")

Можно узнать находится ли таблица в кэше:

In [ ]:
spark.catalog.isCached("taxi")

In [ ]:
spark.sql("SELECT count(*) FROM taxi").show()

Можно очистить весь кэш при помощи `clearCache` и убрать все таблицы из кэша:

In [ ]:
spark.catalog.clearCache()

## Инвалидация кеша

Spark прекрасно справляется с задачей инвалидиации кеша:

Создать parquet файл с 10 строками:

In [ ]:
taxi_df.limit(10).write.mode("overwrite").parquet("/tmp/taxi_cache_invalidate")

In [ ]:
taxi_invalidate_cache_df = spark.read.parquet("/tmp/taxi_cache_invalidate")
cached_taxi_df = taxi_invalidate_cache_df.cache()

In [ ]:
count_df = cached_taxi_df.select(F.count(F.lit(1)))

In [ ]:
count_df.explain()

In [ ]:
count_df.show()

Обновить источник. Создать parquet файл из 10000 строк:

In [ ]:
taxi_df.limit(10000).write.mode("overwrite").parquet("/tmp/taxi_cache_invalidate")

In [ ]:
count_df.explain()

План запроса не изменился (за исключением служебных имен колонок):

![](../imgs/spark-cache-invalidate-plans.drawio.svg)

In [ ]:
count_df.show()

Объем закешированных данных увеличился (ожидаемо):

![](../imgs/spark-cache-invalidate-storage.drawio.svg)

### Вывод

Инвалиация кеша выполняется прозрачно: Apache Spark отслеживает источники данных и при их изменении инвалидирует кеш.

## Кэширование плана

В ситуации, когда план становится очень длинным, есть риск, что случится `StackOverflowError` или драйвер будет тратить очень много времени на построение плана, или другие проблемы во время работы. План можно разбить на несколько частей, и обратиться к ним, когда возникнет необхоимость. Этого можно достичь при помощи механизма Checkpoint. При помощи Checkpoint можно разбить план и материализовать запрос. При обработке следующих трансформаций, Spark начнет создавать новый план.

In [ ]:
spark.sparkContext.setCheckpointDir("plan/checkpoint")

In [ ]:
small_taxi_df = taxi_df.limit(10)

In [ ]:
small_taxi_df = small_taxi_df.checkpoint()

In [ ]:
df = small_taxi_df.select(small_taxi_df.passenger_count)
df.explain()

![](../imgs/spark-scan-checkpointed.drawio.svg)

Обратите внимание:

1. в отличии от `cache`, плане запроса отсутствует информация о том, что был просканирован какой-то parquet файл перед `checkpoint`'ом, т.е. фактически произошло отсечение линии жизни/плана RDD;
2. на диск был сохранен RDD со всеми колонками из parquet файла, а в результате требуется только одна колонка. Следовательно, необходимо применять те же размышления, что и при кешировании, и сохранять только нужные колонки.

In [ ]:
df.show()

В отличии от кеширования, оптимизатор не перенаправляет запросы к сохраненному RDD:

In [ ]:
df = taxi_df \
  .limit(10) \
  .select(taxi_df.passenger_count)

In [ ]:
df.explain()

In [ ]:
df.show()

### Выводы

1. Checkpoint позволяет разбить большой план на несколько частей, каждая из которых может запускает свои задачи и задания на кластере;
1. В отличии от кеширования Checkpoint не показывает какие операции привели к созданию RDD, т.е. происходит отсечение плана;
1. Необходимо выполнять checkpoint с минимально необходимым объемом данных как по количеству колонок, так и по количеству строк, т.к. правила оптимизации **Pushed Filters** и **Pushed Projections** не применяются.

### Задание

Изменить запрос, чтобы сохранить (checkpoint) только нужные колонки и строки:
```python
cp_df = taxi_df.checkpoint()
many_passengers_df = cp_df.select(cp_df.passenger_count, cp_df.trip_distance).where(cp_df.trip_distance > 10)
generous_rides_df = cp_df.where(cp_df.tip_amount / cp_df.total_amount > 0.3).select(cp_df.VendorID, cp_df.trip_distance)
```